# Exploratory Data Analysis with Pandas

This notebook demonstrates essential exploratory data analysis (EDA) techniques using Pandas.

## Topics Covered:
- Dataset overview and inspection
- Missing value analysis
- Statistical summaries
- Groupby operations and aggregations
- Correlation analysis
- Filtering and sorting
- Date/time operations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 2)

## 1. Creating a Sample Dataset

We'll create a synthetic sales dataset with multiple columns including dates, categories, and numerical values.

In [ ]:
# Generate synthetic sales data
n_records = 1000

# Date range
dates = pd.date_range(start='2023-01-01', end='2023-12-31', periods=n_records)

# Product categories
categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Home & Garden']
category_list = np.random.choice(categories, n_records)

# Regions
regions = ['North', 'South', 'East', 'West']
region_list = np.random.choice(regions, n_records)

# Sales amounts (with some variation by category)
base_sales = np.random.uniform(10, 500, n_records)
category_multipliers = {'Electronics': 2.0, 'Clothing': 1.2, 'Food': 0.8, 'Books': 1.0, 'Home & Garden': 1.5}
sales = [base_sales[i] * category_multipliers[category_list[i]] for i in range(n_records)]

# Quantities
quantities = np.random.randint(1, 20, n_records)

# Customer IDs
customer_ids = np.random.randint(1000, 2000, n_records)

# Create DataFrame
df = pd.DataFrame({
    'date': dates,
    'customer_id': customer_ids,
    'category': category_list,
    'region': region_list,
    'sales_amount': sales,
    'quantity': quantities
})

# Calculate unit price
df['unit_price'] = df['sales_amount'] / df['quantity']

# Introduce some missing values (realistic scenario)
missing_indices = np.random.choice(df.index, size=50, replace=False)
df.loc[missing_indices, 'sales_amount'] = np.nan

missing_indices_2 = np.random.choice(df.index, size=30, replace=False)
df.loc[missing_indices_2, 'quantity'] = np.nan

print("Sample dataset created successfully!")
df.head(10)

## 2. Dataset Overview

First, let's understand the basic structure and characteristics of our dataset.

In [ ]:
# Shape of the dataset
print(f"Dataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

In [ ]:
# Column names and data types
print("\nColumn names:")
print(df.columns.tolist())

print("\nData types:")
print(df.dtypes)

In [ ]:
# Comprehensive information about the DataFrame
df.info()

## 3. Missing Value Analysis

Identifying and handling missing values is a critical step in EDA.

In [ ]:
# Count missing values per column
missing_counts = df.isnull().sum()
print("Missing values per column:")
print(missing_counts)

# Percentage of missing values
missing_percentages = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of missing values:")
print(missing_percentages)

In [ ]:
# Visualize missing values
missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': missing_counts.values,
    'Missing_Percentage': missing_percentages.values
})
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]
print("Summary of columns with missing values:")
print(missing_summary)

In [ ]:
# Handle missing values - create a copy for demonstration
df_filled = df.copy()

# Fill missing sales_amount with median
df_filled['sales_amount'].fillna(df_filled['sales_amount'].median(), inplace=True)

# Fill missing quantity with mode (most common value)
df_filled['quantity'].fillna(df_filled['quantity'].mode()[0], inplace=True)

# Recalculate unit_price
df_filled['unit_price'] = df_filled['sales_amount'] / df_filled['quantity']

print("Missing values after filling:")
print(df_filled.isnull().sum())

## 4. Statistical Summaries

Understanding the distribution and central tendencies of numerical columns.

In [ ]:
# Descriptive statistics for numerical columns
df_filled.describe()

In [ ]:
# Detailed statistics for a specific column
print("Statistics for sales_amount:")
print(f"Mean: {df_filled['sales_amount'].mean():.2f}")
print(f"Median: {df_filled['sales_amount'].median():.2f}")
print(f"Standard Deviation: {df_filled['sales_amount'].std():.2f}")
print(f"Min: {df_filled['sales_amount'].min():.2f}")
print(f"Max: {df_filled['sales_amount'].max():.2f}")
print(f"25th Percentile: {df_filled['sales_amount'].quantile(0.25):.2f}")
print(f"75th Percentile: {df_filled['sales_amount'].quantile(0.75):.2f}")
print(f"IQR: {df_filled['sales_amount'].quantile(0.75) - df_filled['sales_amount'].quantile(0.25):.2f}")

In [ ]:
# Summary statistics for categorical columns
print("Category distribution:")
print(df_filled['category'].value_counts())

print("\nRegion distribution:")
print(df_filled['region'].value_counts())

## 5. Groupby Operations and Aggregations

Analyzing data by groups to identify patterns and trends.

In [ ]:
# Group by category and calculate aggregates
category_summary = df_filled.groupby('category').agg({
    'sales_amount': ['sum', 'mean', 'count'],
    'quantity': ['sum', 'mean'],
    'unit_price': 'mean'
}).round(2)

print("Summary by Category:")
print(category_summary)

In [ ]:
# Group by region
region_summary = df_filled.groupby('region').agg({
    'sales_amount': ['sum', 'mean'],
    'customer_id': 'nunique'  # Count unique customers
}).round(2)

region_summary.columns = ['Total Sales', 'Avg Sales', 'Unique Customers']
print("\nSummary by Region:")
print(region_summary)

In [ ]:
# Multiple groupby - category and region
multi_group = df_filled.groupby(['category', 'region'])['sales_amount'].agg(['sum', 'mean', 'count']).round(2)
print("\nSummary by Category and Region:")
print(multi_group.head(10))

## 6. Correlation Analysis

Examining relationships between numerical variables.

In [ ]:
# Select only numerical columns for correlation
numerical_cols = df_filled.select_dtypes(include=[np.number]).columns
correlation_matrix = df_filled[numerical_cols].corr()

print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
# Correlation with a specific column
print("\nCorrelation with sales_amount:")
print(correlation_matrix['sales_amount'].sort_values(ascending=False))

## 7. Filtering and Sorting

Essential techniques for data exploration and analysis.

In [ ]:
# Filter: High-value sales (> 500)
high_value_sales = df_filled[df_filled['sales_amount'] > 500]
print(f"Number of high-value sales: {len(high_value_sales)}")
print(high_value_sales.head())

In [ ]:
# Multiple conditions: Electronics in North region
electronics_north = df_filled[(df_filled['category'] == 'Electronics') & (df_filled['region'] == 'North')]
print(f"\nElectronics sales in North region: {len(electronics_north)}")
print(electronics_north.head())

In [ ]:
# Sort by sales_amount (descending)
top_sales = df_filled.sort_values('sales_amount', ascending=False).head(10)
print("\nTop 10 sales:")
print(top_sales[['date', 'category', 'region', 'sales_amount', 'quantity']])

In [ ]:
# Value counts for categorical analysis
print("\nTop categories by number of transactions:")
print(df_filled['category'].value_counts())

print("\nCategory proportions:")
print(df_filled['category'].value_counts(normalize=True))

## 8. Date/Time Operations

Working with temporal data to identify trends and patterns.

In [ ]:
# Extract date components
df_filled['year'] = df_filled['date'].dt.year
df_filled['month'] = df_filled['date'].dt.month
df_filled['day'] = df_filled['date'].dt.day
df_filled['day_of_week'] = df_filled['date'].dt.day_name()
df_filled['quarter'] = df_filled['date'].dt.quarter

print("Date components added:")
print(df_filled[['date', 'year', 'month', 'day', 'day_of_week', 'quarter']].head())

In [ ]:
# Monthly sales trend
monthly_sales = df_filled.groupby('month')['sales_amount'].agg(['sum', 'mean', 'count']).round(2)
monthly_sales.columns = ['Total Sales', 'Avg Sales', 'Transactions']
print("\nMonthly Sales Summary:")
print(monthly_sales)

In [ ]:
# Sales by day of week
day_of_week_sales = df_filled.groupby('day_of_week')['sales_amount'].agg(['sum', 'mean', 'count']).round(2)
print("\nSales by Day of Week:")
print(day_of_week_sales)

In [ ]:
# Quarterly performance
quarterly_sales = df_filled.groupby('quarter')['sales_amount'].agg(['sum', 'mean', 'count']).round(2)
quarterly_sales.columns = ['Total Sales', 'Avg Sales', 'Transactions']
print("\nQuarterly Sales Summary:")
print(quarterly_sales)

## 9. Advanced Filtering and Insights

Combining techniques to derive actionable insights.

In [ ]:
# Find top-performing category-region combinations
top_combinations = df_filled.groupby(['category', 'region'])['sales_amount'].sum().sort_values(ascending=False).head(10)
print("Top 10 Category-Region Combinations by Total Sales:")
print(top_combinations)

In [ ]:
# Identify customers with highest total purchases
top_customers = df_filled.groupby('customer_id').agg({
    'sales_amount': 'sum',
    'date': 'count'
}).sort_values('sales_amount', ascending=False).head(10)
top_customers.columns = ['Total Purchases', 'Number of Transactions']
print("\nTop 10 Customers:")
print(top_customers)

In [ ]:
# Calculate category-wise market share
category_sales = df_filled.groupby('category')['sales_amount'].sum()
category_market_share = (category_sales / category_sales.sum() * 100).round(2)
print("\nMarket Share by Category (%):")
print(category_market_share.sort_values(ascending=False))

## Summary

In this notebook, we covered essential EDA techniques:

1. **Dataset Overview**: Understanding shape, columns, and data types
2. **Missing Value Analysis**: Identifying and handling missing data
3. **Statistical Summaries**: Mean, median, standard deviation, quantiles
4. **Groupby Operations**: Aggregating data by categories
5. **Correlation Analysis**: Examining relationships between variables
6. **Filtering and Sorting**: Extracting specific subsets of data
7. **Date/Time Operations**: Temporal analysis and trends
8. **Advanced Insights**: Combining techniques for business intelligence

These techniques form the foundation of data analysis and are essential for understanding your data before modeling.